# Análise Exploratória do portal de dados DATA.RIO

## Importação das bibliotecas

In [1]:
import basedosdados as bd
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

## Funções auxiliares

In [3]:
def generate_df(query):
    return bd.read_sql(query, billing_project_id=os.getenv('ID_PROJETO'))

## Carregando dataframes para as questões 1 a 5

Carrega a tabela de Chamados abertos na data 01/04/2023 em um dataframe. Além do filtro por 'data_inicio', também aplicamos o filtro pela 'data_particao', a fim de diminuir a quantidade processada.

In [5]:
query_chamados = """
    SELECT * FROM datario.administracao_servicos_publicos.chamado_1746
    WHERE DATE(data_inicio) = '2023-04-01' AND data_particao = '2023-04-01';
"""

df_chamados = generate_df(query_chamados)

Downloading: 100%|██████████| 73/73 [00:01<00:00, 72.59rows/s]


Carrega a tabela de Bairros em um dataframe.

In [6]:
query_bairros = """
    SELECT * FROM datario.dados_mestres.bairro;
"""

df_bairros = generate_df(query_bairros)

Downloading: 100%|██████████| 164/164 [00:11<00:00, 14.16rows/s]


## Questões 1 a 5

Como o dataframe "df_chamados" já é fruto de uma consulta da tabela de chamados filtrada pela data desejada em todas as questões (01/04/2023), não será necessário aplicar uma filtragem por esta data em cada etapa.

### 1. Quantos chamados foram abertos no dia 01/04/2023?

Tendo em vista que o dataframe "df_chamados" já está filtrado pela data em questão, apenas se faz necessário contarmos quantas linhas ele possui. Deste modo, ao utilizarmos a função len no dataframe, conseguimos a resposta desejada.

In [7]:
len(df_chamados)

73

Resposta: 73 chamados

### 2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?


Para conseguirmos saber o tipo de chamado que teve mais chamados aberto no dia em questão, basta realizarmos a contagem de valores agregando a coluna tipo, e após isso selecionarmos o primeiro valor. Assim, filtrando o dataframe "df_chamados" pela coluna "tipo" e utilizando a função "value_counts", agregamos o dataframe pela coluna desejada e fazemos a contagem de linhas referentes a cada tipo. Por fim, a função "head" com o parâmetro "1", nos retorna o maior valor dessa filtragem.

In [8]:
df_chamados['tipo'].value_counts().head(1)

Poluição sonora    24
Name: tipo, dtype: int64

Resposta: O tipo de chamado com mais chamados abertos no dia 01/04/2023 foi Poluição Sonora, com 24 chamados abertos.

### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?


Para encontrarmos a resposta dessa questão, precisamos relacionar as tabelas de chamados e bairros, a fim de contarmos quais bairros possuem mais chamados abertos na data em questão. Assim, ao fazermos o "merge" dos dataframes "df_chamados" e "df_bairros", utilizando a coluna "id_bairro" presente em ambos, conseguimos um dataframe que contem os dados dos chamados abertos no dia 01/04/2023 com as informações referentes ao bairro do chamado. Logo, basta filtrar da mesma maneira mostrada na questão anterior, porém mudando o tipo para "nome", que é a coluna referente ao nome do bairro, e colocar o parâmentro da função "head" para "3", pegando assim os 3 bairros com mais chamados abertos no dia em questão.  

In [9]:
df_chamados_bairro = df_chamados.merge(df_bairros, on="id_bairro")

In [10]:
df_chamados_bairro['nome'].value_counts().head(3)

Engenho de Dentro    8
Leblon               6
Campo Grande         6
Name: nome, dtype: int64

Respostas: Os 3 bairros com mais chamados abertos na data em questão foram: Engenho de Dentro, com 8 chamados; Leblon, com 6 chamados; e Campo Grande, também com 6 chamados.

### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

Para a resposta em questão, utilizaremos a mesma relação criada na resposta anterior, porém buscamos o nome da subprefeitura com mais chamados abertos no dia em questão, não os 3 bairros com mais chamados abertos nesse dia. Assim, basta mudarmos a filtragem da coluna "nome" para a coluna "subprefeitura", e mudarmos o parâmetro da função "head" para "1", conseguindo assim a subprefeitura com mais chamados abertos nesse dia.

In [11]:
df_chamados_bairro['subprefeitura'].value_counts().head(1)

Zona Norte    25
Name: subprefeitura, dtype: int64

Resposta: A subprefeitura com mais chamados abertos nesse dia foi a Zona Norte, com 25 chamados.

###  5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

Para essa questão, é necessário apenas contar quais registros de chamados no dataframe "df_chamados" não possuem valor na coluna "id_bairro", visto que isso impossibilita-os de terem um bairro associado. Assim, precisamos apenas utilizar a função "loc", filtrando a coluna "id_bairro" com a função "isna", que retorna os registros com valor nulo na coluna em questão. A partir disto, podemos inferir o motivo do chamado não ter bairro associado analisando as suas colunas. 

In [12]:
df_chamados_sem_bairro = df_chamados.loc[df_chamados['id_bairro'].isna()]

In [13]:
df_chamados_sem_bairro

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry
0,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,<NA>,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,<NA>,D,F,No prazo,Encerrado,Atendido parcialmente,None,0,2023-04-01,None


In [14]:
df_chamados_sem_bairro[['id_chamado', 'id_bairro', 'tipo', 'subtipo']]

,id_chamado,id_bairro,tipo,subtipo
0,18516246,None,Ônibus,Verificação de ar condicionado inoperante no ô...


Resposta: Existe 1 chamado que não possui id_bairro (id_chamado 18516246), logo, não possui bairro associado. Isso se deve pelo fato de ser um chamado do tipo "Ônibus", sendo assim, não possui necessariamente uma localização fixa para o chamado.


## Carregando dataframes para as questões de 6 a 10

Carrega a tabela de Eventos em um dataframe.

In [31]:
query_eventos = """
    SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos;
"""

df_eventos = generate_df(query_eventos)

Downloading: 100%|██████████| 4/4 [00:00<00:00, 10.06rows/s]


In [32]:
df_eventos

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


Carrega a tabela de Chamados abertos com o subtipo "Perturbação do sossego" em um dataframe. Além do filtro por subtipo, também aplicamos o filtro pela 'data_particao', a fim de diminuir a quantidade processada. Como sabemos que os eventos ocorreram exclusivamente nos anos de 2022 e 2023, o filtro aplicado pela 'data_particao' busca todos os registros realizados nos dois anos.

In [33]:
query_chamados = """
    SELECT * FROM datario.administracao_servicos_publicos.chamado_1746
    WHERE subtipo = 'Perturbação do sossego' AND data_particao BETWEEN '2022-01-01' AND '2023-12-01';
"""

df_chamados = generate_df(query_chamados)

Downloading: 100%|██████████| 42408/42408 [00:14<00:00, 2853.89rows/s]


## Questões 6 a 10

Como o dataframe "df_chamados" já é fruto de uma consulta da tabela de chamados filtrada pelo subtipo (Perturbação do sossego) e datas desejadas (2022 e 2023) em todas as questões, não será necessário aplicar uma filtragem por data e subtipo em cada etapa.

###  6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

Tendo em vista que o dataframe "df_chamados" já está filtrado pela data em questão, apenas se faz necessário contarmos quantas linhas ele possui. Deste modo, ao utilizarmos a função len no dataframe, conseguimos a resposta desejada.

In [34]:
len(df_chamados)

42408

Resposta: 42408 chamados.

###  7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

Para encontrarmos a resposta dessa questão, precisamos relacionar as tabelas de chamados e eventos, a fim de contarmos quantos chamados foram abertos durante a duração dos eventos em questão. Assim, precisamos fazer o 'merge' dos dataframes 'df_chamados' e 'df_eventos', porém, como queremos relacionar uma coluna do dataframe 'df_chamados' ('data_inicio'), com mais de uma coluna do dataframe 'df_eventos' (um intervalo proposto pelas colunas 'data_inicial' e 'data_final'), não conseguirmos fazer um 'merge' diretamente por essas colunas. Logo, precisamos fazer um produto cartesiano entre os dois dataframes, utilizando o tipo de 'merge' chamado 'cross', retornando assim todas as combinações de linhas entre ambos dataframes. Por fim, basta filtrar o resultado da operção pelas datas desejadas, utilizando o mesmo método da questão anterior.

In [35]:
df_chamados_cross = df_chamados.merge(df_eventos, how='cross')

df_chamados_evento = df_chamados_cross.loc[(df_chamados_cross['data_inicio'].dt.date >= df_chamados_cross['data_inicial']) & (df_chamados_cross['data_inicio'].dt.date <= df_chamados_cross['data_final'])]

In [36]:
df_chamados_evento[['id_chamado', 'data_inicio', 'evento']]

,id_chamado,data_inicio,evento
28017,18078140,2022-12-30 20:18:43,Reveillon
28057,18077988,2022-12-30 18:54:32,Reveillon
28061,18077970,2022-12-30 18:43:04,Reveillon
28073,18077984,2022-12-30 18:53:12,Reveillon
28421,18078946,2022-12-31 11:52:48,Reveillon
...,...,...,...
114489,18080335,2023-01-01 21:55:10,Reveillon
114493,18080337,2023-01-01 21:59:43,Reveillon
114497,18080341,2023-01-01 22:04:48,Reveillon
114525,18079926,2023-01-01 13:54:17,Reveillon


In [37]:
len(df_chamados_evento)

1212

Resposta: 1212 chamados.

### 8. Quantos chamados desse subtipo foram abertos em cada evento?

Como já temos um dataframe que contém a relação entre os chamados e eventos que ocorriam na sua abertura ("df_chamados_evento"), basta fazermos a contagem de cada registro de chamado agrupando por evento. Deste modo, apenas se faz necessário filtrar o dataframe mecionado pela sua coluna de "evento" e utilizar a função "value_counts".

In [38]:
df_chamados_evento['evento'].value_counts()

Rock in Rio    834
Carnaval       241
Reveillon      137
Name: evento, dtype: int64

Resposta: Reveillon - 137 chamados; Carnaval - 241 chamados; Rock in Rio - 834 chamados.

### 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?


A fim de analisarmos qual evento teve a maior média diária de chamados abertos do subtipo "Perturbação do sossego", é necessário relacionarmos a duração de cada evento com o número de chamados do subtipo desejado para cada evento.
Primeiramente, foi criado um dataframe chamado 'df_total_chamados_evento', contendo os eventos e o número de chamados que esse evento recebeu, com base na resposta da questão anterior. Após isso, foi criada a coluna 'duracao' no dataframe 'df_eventos', a fim de gerar a duração para cada registro da tabela de eventos, e com essa informação, foi criado o dataframe 'df_duracao_evento', que possui a informação do nome e duração para cada evento.
Por fim, foi feita a associação entre os dataframes 'df_total_chamados_evento' e 'df_duracao_evento', gerando o dataframe 'df_media_diaria_evento', adicionando uma coluna criada a partir da divisão da  quantidade de chamados e duração de cada evento, a coluna 'media_diaria_evento'.

In [39]:
df_total_chamados_evento = df_chamados_evento['evento'].value_counts().rename_axis('evento').reset_index(name='total_chamados_evento')

In [40]:
df_eventos['duracao'] = (df_eventos['data_final'] - df_eventos['data_inicial']).dt.days + 1
df_duracao_evento = df_eventos[['evento', 'duracao']].groupby(['evento'], as_index=False).sum()

In [41]:
df_media_diaria_evento = df_total_chamados_evento.merge(df_duracao_evento, on='evento')

In [42]:
df_media_diaria_evento['media_diaria_evento'] = round(df_media_diaria_evento['total_chamados_evento']/df_media_diaria_evento['duracao'], 2)

In [43]:
df_media_diaria_evento[['evento', 'media_diaria_evento']]

,evento,media_diaria_evento
0,Rock in Rio,119.14
1,Carnaval,60.25
2,Reveillon,45.67


Resposta: Rock in Rio, com uma média diária de 119.14 chamados.


### 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.


Para conseguirmos realizar essa análise, é necessário utilizar o resultado encontrado na questão anterior, juntamente à média diária de chamados para o período citado. 
A fim de encontrar a média diária para o período de 01/01/2022 até 31/12/2023, precisamos da quantidade de chamados e da duração do período. A quantidade de chamados foi encontrada previamente na questão 6, logo, basta repetirmos o método utilizado nela. Para a duração, basta fazermos a diferença da data final e inicial do período em questão. Com ambas medidas, conseguimos a média diária de chamados abertos desse subtipo no perído desejado dividindo o total de chamados pela duração, e atribuimos esse valor à variável 'media_diaria_periodo'.
Por fim, adicionamos ao dataframe 'df_media_diaria_evento', encontrado na questão anterior, uma nova coluna chamada 'media_diaria_periodo', com o valor da variável 'media_diaria_periodo'. Deste modo, podemos comparar as médias pedidas no enunciado. Para tal, foi calculada a porcentagem de variação entre as médias, gerando assim uma análise comparativa entre os quantitativos desejados.

In [44]:
data_inicial = datetime(2022, 1, 1).date()
data_final = datetime(2023, 12, 31).date()

total_chamados_periodo = len(df_chamados)
duracao_periodo = (data_final - data_inicial).days + 1
media_diaria_periodo = round(total_chamados_periodo / duracao_periodo, 2)

In [45]:
df_media_diaria_evento['media_diaria_periodo'] = media_diaria_periodo

In [46]:
df_media_diaria_evento['porcentagem_variacao'] = round((df_media_diaria_evento['media_diaria_evento']*100)/df_media_diaria_evento['media_diaria_periodo'] - 100, 2)

In [47]:
df_media_diaria_evento[['evento', 'media_diaria_evento', 'media_diaria_periodo', 'porcentagem_variacao']]

,evento,media_diaria_evento,media_diaria_periodo,porcentagem_variacao
0,Rock in Rio,119.14,58.09,105.10
1,Carnaval,60.25,58.09,3.72
2,Reveillon,45.67,58.09,-21.38


Resposta: A média diária de chamados no período de 01/01/2022 até 31/12/2023 foi de 58.09 chamados, a média do Reveillon foi de 45.67 chamados diários, a do Carnaval foi de 60.25 chamados diários e a do Rock in Rio foi de 119.14 chamados diários. Assim, ao compararmos as médias temos que: a média diária do Carnaval foi 3.72% maior que a média diária anual; a média diária do Reveillon foi 21.38% menor que a média diária anual; e a média diária do Rock in Rio foi 105.1% maior que a média diária anual. 
